In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import re

# Declaring constants
USER_EMAIL = "deathstorm226@gmail.com"
USERNAME = "@deathstorm226"
USER_PASSWORD = "Mohd@78600"

# Loading the profile links from the CSV File
links = []
with open('twitter_links.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        links.append(row[0])
print(f"Total number of links: {len(links)}")

# Initializing the Web Driver
driver = webdriver.Chrome()
driver.get("https://twitter.com/i/flow/login")
time.sleep(10)

# Login process
input_field = driver.find_element(By.XPATH, "//input")
input_field.send_keys(USER_EMAIL)
input_field.send_keys(Keys.ENTER)
time.sleep(10)

try:
    username_field = driver.find_element(By.XPATH, "//input[@data-testid='ocfEnterTextTextInput']")
    username_field.send_keys(USERNAME)
    username_field.send_keys(Keys.ENTER)
    time.sleep(5)
except:
    pass

pwd_field = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@name='password']"))
)
pwd_field.send_keys(USER_PASSWORD)
pwd_field.send_keys(Keys.ENTER)
time.sleep(10)

dict_list = []

# Looping through each link
for i, link in enumerate(links):
    driver.get(link)
    time.sleep(5)

    try:
        username_div = driver.find_element(By.XPATH, "//*[contains(@data-testid, 'UserName')]")
    except:
        print(f"Invalid Profile Link. Skipping link: {i+1}")
        continue
    username_span = username_div.find_element(By.XPATH, ".//span")
    username = username_span.text

    # Fetching bio if exists
    try:
        bio_div = driver.find_element(By.XPATH, "//div[@data-testid='UserDescription']")
        bio_span = bio_div.find_element(By.XPATH, ".//span")
        bio = bio_span.text
    except:
        bio = None

    # Fetching website if exists
    try:
        website_anchor = driver.find_element(By.XPATH, "//a[@data-testid='UserUrl']")
        website = website_anchor.get_attribute("href")
    except:
        website = None

    # Fetching location if exists
    try:
        location_span = driver.find_element(By.XPATH, "//span[@data-testid='UserLocation']")
        location = location_span.text
    except:
        location = None

    # Function to extract numbers with 'K', 'M', or 'B'
    def extract_number(text):
        match = re.search(r"([\d,.]+[KM]?)", text)  # Extracts numbers with optional 'K' or 'M'
        return match.group(1) if match else None

    # Fetching following count
    try:
        following_anchor = driver.find_elements(By.XPATH, "//a[contains(@href, 'following')]")[0]
        following = extract_number(following_anchor.text)  # Extracting only numbers with suffix
    except:
        following = None

    # Fetching followers count
    try:
        follower_anchor = driver.find_elements(By.XPATH, "//a[contains(@href, 'followers')]")[0]
        follower = extract_number(follower_anchor.text)  # Extracting only numbers with suffix
    except:
        follower = None

    # Appending profile details to the list
    profile_dict = {
        "Username": username,
        "Bio": bio,
        "Location": location,
        "Following": following,
        "Follower": follower,
        "Website": website
    }
    dict_list.append(profile_dict)
    print(f"Scraped link: {i+1}")

driver.quit()

csv_filename = "profile.csv"

# Dumping the list of dictionaries to CSV file
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=dict_list[0].keys())
    writer.writeheader()
    writer.writerows(dict_list)

print(f"Scraped data saved to {csv_filename}")


In [ ]:
# this code using selenium which automatically put it means scraping data 